Author: Carlos Roberto de Melo

Date: 05/27/2020

Obj: Perform the modeling of spectrum based on pPXF method by Cappellari,and obtein the kinematics./Realizar o modelamento de espectros baeado no código pPXF de Cappellari, e obter a cinemática.

In [7]:
import glob
from os import path

from astropy.io import fits
from scipy import ndimage
import numpy as np

import ppxf as ppxf_package
from ppxf.ppxf import ppxf
import ppxf.ppxf_util as util

import matplotlib.pyplot as plt

In [8]:
frist = np.loadtxt('Output_Ppxf Frist Run.txt')

Adaptation of the Ppxf code to apply different spectra.
**x** and **y** are positions in px, which serves as counter and also index to open the spectrum files, which were saved in the format **y_px X x_px.fits**.

As discussed in the README, the .fits files containing the spectrum have three extensions. Note that pyton, by default, starts a count of zero.

It should be viewed as units of wavelength and flux. By default, pPXF works with wavelength in Angstrons, as we are using the MILES star library. In our case, as we normalize the spectrum, as units of intensity should not cause problems.



Adapatação do código Ppxf para se aplicar a diversos espectros.
x e y são as posições em px, de modo que servem de contador e também de index para abrir os arquivos de espectro, que foram salvos no formato **y_px X x_px.fits**.

Como discutido no README, os arquivos .fits que contém os espectros possuem três extensões. Note que o pyton, por padrão, começa a contagem do zero.

Deve-se antentar as unidades de comprimento de onda e fluxo. Por padrão, o ppxf trabalha com comprimento de onda em Angstrons pois estamos utilizando a biblioteca estelar do MILES. No nosso caso como vamos normalizar o espectro, as unidades de intensidade não devem gerar problemas.



In [9]:
ppxf_dir = '/home/carlos/Desktop/ppxf' #Directory of stellar library/Diretória da biblioteca estelar
cont = 0
for y in range(20):
    for x in range(20):
        chi2_dof = frist[cont][4]
    #READING THE SPECTRUM/LENDO UM ESPECTRO
        file = ('/home/carlos/Desktop/ESO325 MUSE-ColletBinning (01-07-20)/Espectros/%dx%d.fits'%(y,x)) #Path and name of the spectrum/Caminho e nome do arquivo do espectro
        hdu = fits.open(file) #Opening the .fits file/Abrindo o arquivo.fits
        gal_lin = hdu[1].data #Read the data/Lendo os dados
        h1 = hdu[1].header    #Read the header/Lendo o header dos dados
        
    #READING THE VARIANCE/LENDO A VARIÂNCIA DO ESPECTRO
        var = hdu[2].data     #Read the variance/Lendo a variância
        h_var = hdu[2].header #Read the variance header/Lendo o header da variância
        
        
        
    #LOG-REBIN THE SPECTRUM AND VARIANCE/LOGARITMIZANDO O ESPECTRO E A VARIÂNCIA
        #SPECTRUM/ESPECTRO
        FWHM_gal = 2.71  #FWHM of telescope in Angstrons/FWHM do telescópio em Angstrons
        lambRange1 = h1['CRVAL1'] + np.array([0.,h1['CDELT1']*(h1['NAXIS1']-1)]) #Generating the wavelengths/Está linha gera os comprimentos de onda
                                #pois a variável gal_lin armazena em um array apenas as intensidades, mas não os comprimentos de onda
        
        
        #De-redshift
        z = 0.035 # Initial estimate of the galaxy redshift
        lambRange1 = lambRange1/(1+z) # Compute approximate restframe wavelength range
        FWHM_gal = FWHM_gal/(1+z)   # Adjust resolution in Angstrom
        #Depois do passo acima, a galáxia passa a estar no restframe, logo é preeciso setar
        z = 0.
        
        
        galaxy, logLam1, velscale = util.log_rebin(lambRange1, gal_lin) #Log-rebin the spectrum/Logaritimizando o espectro da galáxia
        median_galaxy = np.median(galaxy) #Mean of spectrum for normalization/Média do espectro, pois vamos normalizá-lo pela média
        galaxy = galaxy/median_galaxy        #Normalizing the spectrum/Normalizando o espectro pela média
        lam = np.exp(logLam1)                #Recovering the wavelength in Ansgstrons/Recuperando o comprimento de onda em unidades do SI
        
        #VARIANCE/VARIÂNCIA
        lamRange_var = h_var['CRVAL1'] + np.array([0.,h_var['CDELT1']*(h_var['NAXIS1']-1)])
        erro = np.sqrt(chi2_dof*var) #1-sigma error from variance/1-sigma erro da variância
        erro, loglam_var, velscale_var = util.log_rebin(lamRange_var, erro)
        noise = erro/median_galaxy         #Normalizing the error/Normalizando o erro        
        
    #READING ONE TEMPLATE FOR DETERMINE THE LENGHT/LENDO UM TEMPLATE PARA SABER O TAMANHO DO VETOR FINAL
        #The ideia is make a empty array to after complete with the informations about the templates of the stellar libary/
                    #A ideia é crirar um array vazio para que mais tarde seja preenchido com as informações de todos os templetes.
        vazdekis = glob.glob(ppxf_dir + '/miles_models/Mun1.30*.fits') 
        FWHM_tem = 2.51  #FWHM of templates in Angstrons/FWHM do templete em Angstrons
        velscale_ratio = 2
        hdu = fits.open(vazdekis[0])
        ssp = hdu[0].data
        h2 = hdu[0].header
        
        #Log_rebin TEMPLATE/LOGARITMIZANDO O TEMPLATE
        lamRange2 = h2['CRVAL1'] + np.array([0., h2['CDELT1']*(h2['NAXIS1'] - 1)])
        sspNew, logLam2, velscale_temp = util.log_rebin(lamRange2, ssp, velscale=velscale/velscale_ratio)
        templates = np.empty((sspNew.size, len(vazdekis)))  #Array with necessary size/Array com o tamanho necessário
    
    #FINAL FWHM FOR GALAXY AND TEMPLATE CONVOLUTION/CALCULANDO A FWHM FINAL PARA REALIZAR A CONVOLUÇÃO DO TEMPLATE E GALÁXIA
        FWHM_dif = np.sqrt(FWHM_gal**2 - FWHM_tem**2)
        sigma = FWHM_dif/2.355/h2['CDELT1'] #New sigma PSF/Novo sigma da PSF.
    
    #READING ALL TEMPLATES/LENDO TODOS OS TEMPLATES E JÁ LOGARITMIZANDO
        for j, file in enumerate(vazdekis):
            hdu = fits.open(file)
            ssp = hdu[0].data
            ssp = ndimage.gaussian_filter1d(ssp, sigma) #Convolution/Realizando a convolução
            sspNew, logLam2, velscale_temp = util.log_rebin(lamRange2, ssp, velscale=velscale/velscale_ratio)
            templates[:, j] = sspNew/np.median(sspNew)
    
    #ADICIONANDO UM SHIFT PARA COINCIDIR OS COMPRIMENTOS DE ONDA DO ESPECTRO E DO TEMPLATE
        #Artificial shift for the galaxy and templates wavelenght may coincide.    
        #Nem sempre os comprimentos de onda do templete e do espectro coincidem, por esse motivo é dado um 
        #shift artificial ao templete, mas que após o ajuste é retirado.
        c = 299792.458 #Speed of light/Velocidade da Luz
        dv = (np.mean(logLam2[:velscale_ratio]) - logLam1[0])*c #Shift
        goodPixels = util.determine_goodpixels(logLam1, lamRange2, z)  #Mask bad emission lines/Máscaras para linhas de emissão
        vel = c*np.log(1 + z)
        start = [vel, 200.] #Inicial guess/Chute inicial
    
    #START OF FIT/INÍCIO DO AJUSTE
       #For each spectrum the modeling is done. After the best solution is printed 
       #(velocity, velocity dispersion, another moments). Then, a table is write with the information about the
       #pixel position, velocity, velocity dispersion, chi²/DOF and 1-sigma error. The table is saved with name
       #'Output_Ppxf.txt'.  
        
       #Para cada espectro ele realiza o ajuste e printa abaixo o nome do px em que foi realizado o ajuste,
       #bem como as infomações do melhor ajuste (qui², Velocidade, Dispersão, Demais momentos). Então cria uma
       #tabela com as informações necessárias e a salva com o nome 'Output_Ppxf.txt'.
        pp = ppxf(templates, galaxy, noise, velscale, start,
                lam = lam, plot=False, moments=2,
                  degree=6,vsyst=dv, goodpixels=goodPixels, velscale_ratio=velscale_ratio)
        #For more informations about the method, visit the paper of Cappellari avaible in README
        #Para mais informações sobre o método, visite o paper de Cappellari disponível no README
        
        #Plot and save the best model/Plota e salva o melhor modelo
        pp.plot() #Plot the fit/Plota o fit
        plt.savefig('/home/carlos/Desktop/ESO325 MUSE-ColletBinning (01-07-20)/Spec Fit Milles Second/%dx%d.png'%(y,x),format='png', dpi=1200)
        plt.clf()
        cont = cont+1
        print(y,x)
        print('Vel :%f.\nDisp:%f.'%(pp.sol[0],pp.sol[1]))
        print('Best-fitting redshift z:', (0.035 + 1)*(1 + pp.sol[0]/c) - 1)
        error = pp.error*np.sqrt(pp.chi2) #1-sigma error/1-sigma erro
    #ESCREVENDO UMA TABELA COM OS DADOS DE Y_PX, X_PX,  VELOCIDADE(KM/S) E DISPERSÃO(KM/S)
        newrow = [np.int(y), np.int(x), pp.sol[0], pp.sol[1], pp.chi2, error[1]]
        if y == 0 and x == 0:
            table = np.array([np.int(y),np.int(x),pp.sol[0], pp.sol[1],pp.chi2, error[1]])
        else:
            table = np.vstack([table,newrow])
        print('\n####################\n')

 Best Fit:       Vel     sigma
 comp.  0:      -343       304
chi2/DOF: 1
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  4 / 150
0 0
Vel :-343.291833.
Disp:303.612839.
Best-fitting redshift z: 0.03381482326285945

####################

 Best Fit:       Vel     sigma
 comp.  0:      -344       244
chi2/DOF: 1
method = capfit; Jac calls: 6; Func calls: 20; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  3 / 150
0 1
Vel :-344.408452.
Disp:243.743870.
Best-fitting redshift z: 0.03381096826110097

####################

 Best Fit:       Vel     sigma
 comp.  0:      -327       392
chi2/DOF: 1
method = capfit; Jac calls: 4; Func calls: 14; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  3 / 150
0 2
Vel :-327.498766.
Disp:392.317691.
Best-fitting redshift z: 0.03386934706334599

####################

 Best Fit:       Vel     sigma
 comp.  0:      -271       321
chi2/DOF: 1
method = capfit; Jac calls: 5; Func ca

1 7
Vel :-292.671557.
Disp:290.393050.
Best-fitting redshift z: 0.03398958411672792

####################

 Best Fit:       Vel     sigma
 comp.  0:      -293       324
chi2/DOF: 1
method = capfit; Jac calls: 4; Func calls: 14; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  2 / 150
1 8
Vel :-292.810392.
Disp:323.525235.
Best-fitting redshift z: 0.03398910480469053

####################

 Best Fit:       Vel     sigma
 comp.  0:      -335       299
chi2/DOF: 1
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  3 / 150
1 9
Vel :-334.925868.
Disp:298.796094.
Best-fitting redshift z: 0.03384370582255314

####################

 Best Fit:       Vel     sigma
 comp.  0:      -325       301
chi2/DOF: 1
method = capfit; Jac calls: 6; Func calls: 20; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  5 / 150
1 10
Vel :-325.438426.
Disp:301.296069.
Best-fitting redshift z: 0.033876460158953625

####################

 Be

2 15
Vel :-330.698429.
Disp:329.802877.
Best-fitting redshift z: 0.0338583005838462

####################

 Best Fit:       Vel     sigma
 comp.  0:      -303       310
chi2/DOF: 1
method = capfit; Jac calls: 5; Func calls: 17; Status: 4
linear_method = nnls; Nonzero Templates (>0.1%):  2 / 150
2 16
Vel :-303.076676.
Disp:309.892750.
Best-fitting redshift z: 0.033953661602557084

####################

 Best Fit:       Vel     sigma
 comp.  0:      -263       308
chi2/DOF: 1
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  2 / 150
2 17
Vel :-263.345107.
Disp:308.126505.
Best-fitting redshift z: 0.03409083041181593

####################

 Best Fit:       Vel     sigma
 comp.  0:      -287       297
chi2/DOF: 1
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  2 / 150
2 18
Vel :-287.086456.
Disp:297.035465.
Best-fitting redshift z: 0.03400886605384956

####################

 

4 3
Vel :-352.580692.
Disp:323.650213.
Best-fitting redshift z: 0.033782754514722235

####################

 Best Fit:       Vel     sigma
 comp.  0:      -305       310
chi2/DOF: 1
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  2 / 150
4 4
Vel :-304.868571.
Disp:310.150081.
Best-fitting redshift z: 0.033947475286070006

####################

 Best Fit:       Vel     sigma
 comp.  0:      -296       301
chi2/DOF: 1
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  3 / 150
4 5
Vel :-295.948787.
Disp:300.852261.
Best-fitting redshift z: 0.03397826984553065

####################

 Best Fit:       Vel     sigma
 comp.  0:      -341       318
chi2/DOF: 1
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  3 / 150
4 6
Vel :-340.509331.
Disp:317.715045.
Best-fitting redshift z: 0.033824429539808465

####################

 B

5 11
Vel :-319.114480.
Disp:340.700058.
Best-fitting redshift z: 0.03389829287431789

####################

 Best Fit:       Vel     sigma
 comp.  0:      -322       349
chi2/DOF: 1
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  4 / 150
5 12
Vel :-322.146400.
Disp:348.718668.
Best-fitting redshift z: 0.03388782551095271

####################

 Best Fit:       Vel     sigma
 comp.  0:      -311       314
chi2/DOF: 1
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  3 / 150
5 13
Vel :-310.623499.
Disp:314.341143.
Best-fitting redshift z: 0.033927607041236874

####################

 Best Fit:       Vel     sigma
 comp.  0:      -310       329
chi2/DOF: 1
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  3 / 150
5 14
Vel :-310.155767.
Disp:329.265553.
Best-fitting redshift z: 0.03392922183329694

####################



...

15 7
Vel :-295.988113.
Disp:317.769088.
Best-fitting redshift z: 0.03397813407516881

####################

 Best Fit:       Vel     sigma
 comp.  0:      -322       327
chi2/DOF: 1
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  3 / 150
15 8
Vel :-322.348924.
Disp:326.551103.
Best-fitting redshift z: 0.03388712631770807

####################

 Best Fit:       Vel     sigma
 comp.  0:      -292       316
chi2/DOF: 1
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  3 / 150
15 9
Vel :-292.326440.
Disp:316.183110.
Best-fitting redshift z: 0.03399077559212116

####################

 Best Fit:       Vel     sigma
 comp.  0:      -324       311
chi2/DOF: 1
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  5 / 150
15 10
Vel :-323.541622.
Disp:310.889900.
Best-fitting redshift z: 0.03388300866238225

####################



16 15
Vel :-309.460984.
Disp:262.364198.
Best-fitting redshift z: 0.0339316204943938

####################

 Best Fit:       Vel     sigma
 comp.  0:      -327       323
chi2/DOF: 1
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  3 / 150
16 16
Vel :-326.852579.
Disp:322.695419.
Best-fitting redshift z: 0.03387157795166473

####################

 Best Fit:       Vel     sigma
 comp.  0:      -336       255
chi2/DOF: 1
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  2 / 150
16 17
Vel :-336.002559.
Disp:254.936701.
Best-fitting redshift z: 0.03383998866707061

####################

 Best Fit:       Vel     sigma
 comp.  0:      -299       279
chi2/DOF: 1
method = capfit; Jac calls: 6; Func calls: 20; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  2 / 150
16 18
Vel :-298.917246.
Disp:278.654901.
Best-fitting redshift z: 0.033968021573042906

###################

18 3
Vel :-325.333097.
Disp:298.632007.
Best-fitting redshift z: 0.033876823793487665

####################

 Best Fit:       Vel     sigma
 comp.  0:      -334       273
chi2/DOF: 1
method = capfit; Jac calls: 6; Func calls: 20; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  4 / 150
18 4
Vel :-333.747160.
Disp:272.655299.
Best-fitting redshift z: 0.03384777518110926

####################

 Best Fit:       Vel     sigma
 comp.  0:      -346       304
chi2/DOF: 1
method = capfit; Jac calls: 6; Func calls: 20; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  3 / 150
18 5
Vel :-345.810046.
Disp:304.042871.
Best-fitting redshift z: 0.03380612941393002

####################

 Best Fit:       Vel     sigma
 comp.  0:      -336       303
chi2/DOF: 1
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  1 / 150
18 6
Vel :-335.727209.
Disp:303.079223.
Best-fitting redshift z: 0.03384093928430065

####################



19 11
Vel :-336.737558.
Disp:288.639526.
Best-fitting redshift z: 0.03383745116464265

####################

 Best Fit:       Vel     sigma
 comp.  0:      -297       268
chi2/DOF: 1
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  4 / 150
19 12
Vel :-296.852867.
Disp:267.525160.
Best-fitting redshift z: 0.03397514860931938

####################

 Best Fit:       Vel     sigma
 comp.  0:      -298       274
chi2/DOF: 1
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  4 / 150
19 13
Vel :-298.493739.
Disp:274.463744.
Best-fitting redshift z: 0.03396948368364017

####################

 Best Fit:       Vel     sigma
 comp.  0:      -266       356
chi2/DOF: 1
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  2 / 150
19 14
Vel :-265.976859.
Disp:356.115927.
Best-fitting redshift z: 0.03408174458089208

###################

<Figure size 432x288 with 0 Axes>

In [10]:
np.savetxt('Output_Ppxf Second Run.txt',table, header= 'y_px x_px vel(km/s) disp(km/s) chi2/DOF 1-sigma', fmt='%i %i %.18e %.18e %.18e %.18e',delimiter=' ') 